<a href="https://colab.research.google.com/github/ragalaa/Machine-Learning-Basics/blob/master/RiskModellingTechniques_VaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from pandas_datareader import data as web
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
assets =  [ 'AAPL','ADBE','CVX','GOOG','IBM','MDLZ','MSFT','NFLX','ORCL','SBUX','^FVX','^GSPC']    #['NXP']
data = pd.DataFrame()
for stock in assets:
    data[stock] = web.DataReader(stock, data_source='yahoo',start='2007-1-1' ,end='2017-1-31')['Adj Close']

# data = pd.read_csv('Portfolio_Data.csv',sep=",")

In [14]:
data = data.sort_values(['Date'], ascending=[True])
returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ['float64','int64']]].pct_change()
returns = returns[1:]
returns['Intercept'] = 1
returns

,AAPL,ADBE,CVX,GOOG,IBM,MDLZ,MSFT,NFLX,ORCL,SBUX,^FVX,^GSPC,Intercept
Date,,,,,,,,,,,,,
2007-01-04,0.022196,0.022545,-0.009722,0.033512,0.010693,-0.000842,-0.001675,-0.047351,0.009709,0.001135,-0.011166,0.001228,1
2007-01-05,-0.007121,-0.004900,0.003842,0.008132,-0.009053,-0.011243,-0.005703,-0.021302,-0.002262,-0.004251,0.008469,-0.006085,1
2007-01-08,0.004938,-0.004185,0.012757,-0.007410,0.015192,-0.006538,0.009784,-0.039500,0.012472,-0.003699,0.003015,0.002220,1
2007-01-09,0.083070,-0.020272,-0.011477,0.003970,0.011830,0.005151,0.001003,0.006714,-0.002240,-0.004285,-0.000644,-0.000517,1
2007-01-10,0.047856,-0.010346,-0.017273,0.008157,-0.011791,0.021918,-0.010014,0.003335,-0.002806,-0.003155,0.003437,0.001940,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-25,0.015921,0.004661,0.007476,0.014323,0.013587,0.003561,0.002519,-0.004211,0.001247,0.004449,0.025813,0.008026,1
2017-01-26,0.000492,-0.011991,-0.005885,-0.004212,0.002075,-0.010645,0.009265,-0.004014,-0.000498,-0.004089,-0.012582,-0.000735,1
2017-01-27,0.000082,0.009833,-0.023681,-0.010623,-0.007612,-0.009191,0.023495,0.025115,0.002492,-0.040027,-0.010703,-0.000866,1


In [0]:
stocknames = [ 'AAPL','ADBE','CVX','GOOG','IBM','MDLZ','MSFT','NFLX','ORCL','SBUX'] 
factornames = ['^FVX','^GSPC','Intercept']

In [0]:
stockreturns = returns[stocknames]
factorreturns = returns[factornames]

In [0]:
weights = np.array([1.0/len(stocknames)]* len(stocknames))


In [26]:
## portfolio variance
"""
p = w1Y1 + w2Y2+ w3Y3+....+wkYk
Var(p) = W *Cov(Y) * W ^T
"""

historicalTotalRisk = np.dot(np.dot(weights,stockreturns.cov()),weights.T)
historicalTotalRisk

0.00018625227892472834

In [0]:
#2. Building Factor Models:
## factors of volatility SP500 and Intrest rates.(FVX,GSPC)
import statsmodels.api as sm

In [29]:
xdata = factorreturns
xdata.head()

,^FVX,^GSPC,Intercept
Date,,,
2007-01-04,-0.011166,0.001228,1
2007-01-05,0.008469,-0.006085,1
2007-01-08,0.003015,0.002220,1
2007-01-09,-0.000644,-0.000517,1
2007-01-10,0.003437,0.001940,1


In [30]:
modelcoeffs = []
for onestockname in stocknames:
    yData = stockreturns[onestockname]
    model = sm.OLS(yData,xdata)
    result = model.fit()
    modelcoeffrow = list(result.params)
    modelcoeffrow.append(np.std(result.resid,ddof=1))
    modelcoeffs.append(modelcoeffrow)
    print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     749.1
Date:                Thu, 27 Feb 2020   Prob (F-statistic):          2.60e-256
Time:                        04:39:53   Log-Likelihood:                 6833.1
No. Observations:                2537   AIC:                        -1.366e+04
Df Residuals:                    2534   BIC:                        -1.364e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
^FVX          -0.0056      0.010     -0.573      0.5

In [34]:
modelcoeffs = pd.DataFrame(modelcoeffs)
modelcoeffs.columns =["B_SP","B_FVX","Alpha","Residvol"]
modelcoeffs["Names"] = stocknames
modelcoeffs

,B_SP,B_FVX,Alpha,Residvol,Names
0,-0.005627,0.964032,0.000902,0.016373,AAPL
1,-0.003033,1.128756,0.000329,0.015245,ADBE
2,-0.004062,1.064116,0.000178,0.010594,CVX
3,-0.009006,0.936995,0.000406,0.014313,GOOG
4,0.009160,0.753905,0.000214,0.009982,IBM
5,-0.014510,0.649082,0.000279,0.010933,MDLZ
6,-0.008697,0.960744,0.000296,0.012656,MSFT
7,-0.015817,0.927033,0.001797,0.033290,NFLX
8,0.000468,1.005277,0.000249,0.012302,ORCL
9,0.005220,1.043412,0.000412,0.015225,SBUX


In [0]:
##3.Portfolio variance
"""

Totalvar(p) = SystematicVar(p) +IdiosyntricVar(p)

SystematicVar = W   *B     *Cov(f)   *B^t   *W^t
1*1             1*k k*2    2*2       2*k    k*1

Idosyntricvar = ei=1 to K Wi^2Var(ei)
"""

factorcov = factorreturns[['^FVX','^GSPC']].cov()
reconstructedcov = np.dot(np.dot(modelcoeffs[["B_FVX","B_SP"]],factorcov),modelcoeffs[["B_FVX","B_SP"]].T)


In [0]:
systemicRisk = np.dot(np.dot(weights,reconstructedcov),weights.T)
IdosyntricRisk = sum(modelcoeffs["Residvol"] * modelcoeffs["Residvol"]*weights*weights)

In [45]:
factorModelTotalRisk = IdosyntricRisk + systemicRisk
factorModelTotalRisk

0.0011568305204948902

In [47]:
# 4. Scenario stress testing:--
## FVX step size of 0.05 ( 5% increase)
# SP500 step size 0.02 (2%)

fvxscenarios = np.arange(min(returns['^FVX']),max(returns['^FVX']),0.05)
fvxscenarios

array([-0.23213383, -0.18213383, -0.13213383, -0.08213383, -0.03213383,
        0.01786617,  0.06786617,  0.11786617,  0.16786617])

In [0]:
spScenarios = np.arange(min(returns['^GSPC']),max(returns['^GSPC']),0.02)

In [0]:
scenarios = []
for oneFVXvalue in fvxscenarios:
    for oneSPvalue in spScenarios:
        onescenario =[oneFVXvalue,oneSPvalue]
        for onestockname in stocknames:
            alpha = float(modelcoeffs[modelcoeffs["Names"]== onestockname]["Alpha"])
            beta_sp = float(modelcoeffs[modelcoeffs["Names"]== onestockname]["B_SP"])
            bets_FVX = float(modelcoeffs[modelcoeffs["Names"]== onestockname]["B_FVX"])
            onestockpredictedReturn = alpha+ beta_sp*oneSPvalue + bets_FVX *oneFVXvalue
            onescenario.append(onestockpredictedReturn)
            scenarios.append(onescenario)

In [0]:
scenarios = pd.DataFrame(scenarios)
scenarios.columns= ['^FVX','^GSPC', 'AAPL','ADBE','CVX','GOOG','IBM','MDLZ','MSFT','NFLX','ORCL','SBUX'] 

In [58]:
scenarioscov = scenarios[stocknames].cov()
scenarioscov

,AAPL,ADBE,CVX,GOOG,IBM,MDLZ,MSFT,NFLX,ORCL,SBUX
AAPL,0.015505,0.018154,0.017115,0.015070,0.012125,0.010440,0.015452,0.014910,0.016168,0.016782
ADBE,0.018154,0.021256,0.020039,0.017645,0.014197,0.012223,0.018092,0.017458,0.018931,0.019649
CVX,0.017115,0.020039,0.018892,0.016635,0.013384,0.011524,0.017056,0.016458,0.017847,0.018524
GOOG,0.015070,0.017645,0.016635,0.014648,0.011785,0.010147,0.015019,0.014492,0.015715,0.016311
IBM,0.012125,0.014197,0.013384,0.011785,0.009483,0.008163,0.012084,0.011659,0.012644,0.013124
MDLZ,0.010440,0.012223,0.011524,0.010147,0.008163,0.007030,0.010404,0.010040,0.010886,0.011299
MSFT,0.015452,0.018092,0.017056,0.015019,0.012084,0.010404,0.015400,0.014860,0.016113,0.016724
NFLX,0.014910,0.017458,0.016458,0.014492,0.011659,0.010040,0.014860,0.014339,0.015548,0.016137
ORCL,0.016168,0.018931,0.017847,0.015715,0.012644,0.010886,0.016113,0.015548,0.016860,0.017500
SBUX,0.016782,0.019649,0.018524,0.016311,0.013124,0.011299,0.016724,0.016137,0.017500,0.018164


In [60]:
scenarioTotalRisk = np.dot(np.dot(weights,scenarioscov),weights.T)
scenarioTotalRisk

0.014846437033500682

In [0]:
## 4. Quantifying the Worst case scenario
"""
Calculating Var(Variance at Risk):

Var = P *Z(zstats) * sigma * squareroot of h
p= $ invested
sigma  = square root of variance of returns( from factor model, or scenario or historical analysis)
z = number of standard deviations away from mean that the propability = 1- sigma
h = number of trading periods
"""

from scipy.stats import norm
import math

In [0]:
def calcVaR(risk,conflevel,principal = 1,nummonths = 1):
    vol = math.sqrt(risk)
    return abs(principal *norm.ppf(1-conflevel,0,1)*vol*math.sqrt(nummonths))

In [64]:
calcVaR(scenarioTotalRisk,0.99)

0.2834560810824161

In [65]:
calcVaR(historicalTotalRisk,0.99)

0.03174866408345266

In [66]:
calcVaR(factorModelTotalRisk,0.99)

0.0791242355510623